# تحلیلگر احساسات

این پروژه از کتابخانه‌های NLP برای پیش‌پردازش داده‌ها، تحلیل آن‌ها و پیش‌بینی نظرات جدید بر اساس داده‌های قبلی (یادگیری ماشین) استفاده می‌کند.  
من یک مجموعه داده 150,000 نمونه‌ای از وبسایت quera.org دریافت کردم. همچنین یک مجموعه داده بزرگتر از kaggle.com به دست آوردم و آن را در ریپازیتوری با نام big_train قرار دادم.

برای آموزش مدل با استفاده از مجموعه‌داده کوچکتر، از کد زیر استفاده کنید:

In [ ]:
train_data = pd.read_csv('train.csv')

اگر سخت‌افزار قدرتمندی برای پردازش مجموعه‌داده بزرگ دارید، می‌توانید از دستور زیر استفاده کنید:

In [ ]:
train_data = pd.read_csv('big_train.csv', usecols=['body', 'recommendation_status'])

این مجموعه‌داده ترکیبی از داده‌های سایت quera.org و مجموعه‌داده سایت Kaggle است.

این پروژه روی سخت‌افزاری با مشخصات زیر اجرا شده است:
2 هسته پردازشی مجازی (vCPU)، 5 گیگابایت حافظه رم

برای مجموعه‌داده اولیه به دقت حدود 60% و برای مجموعه‌داده بزرگتر به دقت حدود 75% دست یافتم.

<hr>

## وارد کردن ابزار ها و کتابخانه های لازم

ابتدا کتابخانه‌های مورد نیاز را وارد می‌کنیم. شما می‌توانید کتابخانه‌های لازم را از طریق فایل requirements.txt موجود در ریپازیتوری نصب کنید.

In [1]:
import pandas as pd
from hazm import Normalizer, word_tokenize, Stemmer, stopwords_list
import re
from tqdm import tqdm
from gensim.models import Word2Vec
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

<hr>

## بارگذاری مجموعه داده

در این مرحله، فایل‌های مجموعه‌داده را می‌خوانیم. کد زیر بر اساس مجموعه‌داده سایت quera.org نوشته شده است. با توجه به قابلیت‌های سخت‌افزاری سیستم شما، می‌توانید کد را برای بارگذاری مجموعه‌داده بزرگتر (big_train.csv) تغییر دهید.

In [3]:
train_data = pd.read_csv('train.csv') 
test_data = pd.read_csv('test.csv')

پس از بارگذاری مجموعه‌داده، می‌توانیم از دستورات زیر برای استخراج اطلاعات و وضعیتی از داده‌ها استفاده کنیم:

In [9]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149400 entries, 0 to 149399
Data columns (total 2 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   body                   149400 non-null  object
 1   recommendation_status  149400 non-null  object
dtypes: object(2)
memory usage: 2.3+ MB


In [5]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   body    600 non-null    object
dtypes: object(1)
memory usage: 4.8+ KB


با آخرین دستور می‌توانید تعداد نظرات "توصیه شده"، "توصیه نشده" و "نظری ندارم" را در مجموعه‌داده مشاهده کنید.

('recommended' , 'not_recommended' , 'no_idea')

In [7]:
train_data['recommendation_status'].value_counts()

recommendation_status
not_recommended    49800
recommended        49800
no_idea            49800
Name: count, dtype: int64

<hr>

## مدیریت داده های از دست رفته و رمزگذاری برچسب ها

در این مرحله، ما:

مقادیر null و NaN را در مجموعه‌داده با مقادیر مناسب (مثلاً no_idea) پر می‌کنیم.

برچسب‌های وضعیت توصیه را به مقادیر عددی برای پردازش آسان‌تر تبدیل می‌کنیم.

In [9]:
# Replace Nan and Null with no_idea Lable
# Convert "recommended" data to 1 and "not_recommended" data to 0.

train_data["recommendation_status"] = train_data["recommendation_status"].fillna("no_idea")

valid_statuses = {"no_idea", "recommended", "not_recommended"}
train_data["recommendation_status"] = train_data["recommendation_status"].apply(
    lambda x: x if x in valid_statuses else "no_idea"
)

train_data["recommendation_status"] = train_data["recommendation_status"].map({
    "no_idea": 2,
    "recommended": 1,
    "not_recommended": 0
})

اکنون باید بررسی کنیم آیا عملیات پیش‌پردازش (مدیریت مقادیر null و کدگذاری برچسب‌ها) به درستی انجام شده است یا خیر.

In [17]:
# checking the values stored in "recommendation_starus"
train_data["recommendation_status"].unique()

array([0, 1, 2])

In [11]:
train_data["recommendation_status"].value_counts()

recommendation_status
0    49800
1    49800
2    49800
Name: count, dtype: int64

<hr>

## تعریف تابع برای پیش پردازش داده ها

در این مرحله، تابعی برای پیش‌پردازش متن ایجاد می‌کنیم که وظایف زیر را انجام می‌دهد:

نرمال‌سازی متن: تبدیل متن به حروف کوچک

توکن‌سازی: تقسیم متن به کلمات یا توکن‌های مجزا

حذف کلمات توقف: حذف کلمات رایج که معنی چندانی ندارند (مثل "و"، "که"، "چون")

ریشه‌یابی: کاهش کلمات به شکل ریشه آنها (مثال: "کتاب‌ها" → "کتاب")

حذف کاراکترهای خاص و اعداد: پاکسازی متن با حذف نمادها و ارقام غیرضروری

In [13]:
# Initialize tools
stopwords = set(stopwords_list())  # Convert to set for faster lookup
normalizer = Normalizer()
stemmer = Stemmer()

# Define regex patterns
punctuations = r'[!()-\[\]{};:\'",؟<>./?@#$%^&*_~]'
numbers_regex = r'[۰-۹\d]+'  # Combined Persian and Latin numbers
white_space = r'\s+'

def preprocess_text(text):
    # Normalize text
    text = normalizer.normalize(str(text))
    
    # Remove numbers and punctuations
    text = re.sub(numbers_regex, '', text)  # Remove all numbers
    text = re.sub(punctuations, ' ', text)  # Replace punctuations with space
    
    # Normalize whitespace
    text = re.sub(white_space, ' ', text).strip()  # Replace multiple spaces with single space
    
    # Tokenize and process tokens
    tokens = word_tokenize(text)
    processed_tokens = [
        stemmer.stem(token)  # Stem each token
        for token in tokens
        if token not in stopwords and token.strip()  # Remove stopwords and empty tokens
    ]
    
    return processed_tokens

اکنون تابع preprocess_text را روی یک نمونه ورودی آزمایش می‌کنیم تا مطمئن شویم به درستی کار می‌کند. خروجی‌های مورد انتظار به شرح زیر است:

['متولد', 'سال', 'هس']

In [19]:
exmpale = "من متولد سال ۱۳۷۷ هستم"
preprocess_text(exmpale)

['متولد', 'سال', 'هس']

حالا که تابع پیش‌پردازش آماده است، آن را روی تمام نظرات موجود در مجموعه‌داده train_data اعمال می‌کنیم. این کار داده‌ها را برای استفاده با مدل Word2Vec آماده می‌کند. نتایج پیش‌پردازش شده را در ستون جدیدی به نام preprocess ذخیره می‌کنیم.

In [21]:
dataes = train_data['body']

def process_chunks(series, chunk_size=1000):
    chunks = [series[i:i + chunk_size] for i in range(0, len(series), chunk_size)]
    processed_data = []
    
    for chunk in tqdm(chunks, desc="Processing chunks"):
        processed_chunk = chunk.apply(preprocess_text)
        processed_data.extend(processed_chunk)
    
    return pd.Series(processed_data)

# Process data in chunks with progress bar
data_processed = process_chunks(dataes)

Processing chunks: 100%|██████████| 150/150 [01:13<00:00,  2.04it/s]


In [23]:
train_data["preprocess"] = data_processed
train_data.head()

,body,recommendation_status,preprocess
0,جنسش‌خوب‌بود‌خیلی‌بدبدبود,0,[جنسش‌خوب‌بود‌خیلی‌بدبدبود]
1,به کار میاد شک ندارم,1,"[کار, میاد, شک, ندار]"
2,چیزی ک توعکسه واست میفرستن ولی هم جنسش خوب نیس...,2,"[ک, توعکسه, واس, میفرستن, جنس, کوچیکتره, صفه, ..."
3,رنگش خیلی خوبه . براق هم هست و زود خشک میشه . ...,2,"[رنگ, خوبه, براق, هس, زود, خشک, میشه, زد, تو, ..."
4,من مرجوع کردم قسمت پاچه شلوار برام تنگ بود ولی...,2,"[مرجوع, قسم, پاچه, شلوار, برا, تنگ, جنس, بد, ن..."


<hr>

## تعبیه داده‌ها با استفاده از Word2Vec

حالا که داده‌ها پیش‌پردازش و در ستون preprocess ذخیره شده‌اند، از الگوریتم Word2Vec برای تبدیل کلمات به بردارهای عددی استفاده می‌کنیم. این مرحله شامل آموزش یک مدل Word2Vec روی داده‌های متنی پیش‌پردازش شده برای ایجاد تعبیه‌های کلمه است.

In [25]:
model = Word2Vec(sentences=train_data["preprocess"], vector_size=100, window=5, min_count=1, workers=4)

حالا که مدل Word2Vec آموزش داده شده است، آن را با یافتن کلماتی که بیشترین شباهت را به کلمه "دوست" دارند آزمایش می‌کنیم. این کار به ما کمک می‌کند تا کیفیت تعبیه‌ها را ارزیابی کنیم و بفهمیم که مدل تا چه حد روابط معنایی را به خوبی یاد گرفته است.

In [27]:
model.wv.most_similar("دوست")

[('دوسشون', 0.9282898902893066),
 ('دوستشون', 0.9159670472145081),
 ('دوس', 0.8406839966773987),
 ('نگم😍', 0.7801008224487305),
 ('کارتخو', 0.773705780506134),
 ('اینوخرید', 0.7618856430053711),
 ('اصرار', 0.7595681548118591),
 ('عاشقشه', 0.7553688287734985),
 ('انتظاربیشتر', 0.7509226202964783),
 ('انتظارشو', 0.7480288147926331)]

در این مرحله، تابعی به نام sentence_vector طراحی می‌کنیم که بردار تعبیه را برای هر نظر با محاسبه میانگین بردارهای کلمات موجود در آن نظر تولید می‌کند. این فرآیند یک بردار ثابت‌اندازه برای هر جمله ایجاد می‌کند که می‌تواند به عنوان ورودی مدل‌های یادگیری ماشین استفاده شود.

In [29]:
# Create sentence vectors by averaging word vectors
def sentence_vector(sentence):
    vectors = []
    for word in sentence:
        try:
            vectors.append(model.wv[word])
        except KeyError:
            # Handle words not in vocabulary (e.g., use a zero vector)
            vectors.append(np.zeros(100))  # Assuming vector_size=100
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(100)

حالا که تابع sentence_vector تعریف شده است، آن را روی ستون preprocess از مجموعه‌داده train_data اعمال می‌کنیم. این کار هر نظر را به بردار جمله مربوطه تبدیل می‌کند. نتایج در متغیری به نام sentence_vectors ذخیره خواهند شد.

In [31]:
sentence_vectors = train_data['preprocess'].apply(sentence_vector)
sentence_vectors

0         [0.008226887, -0.006862732, 0.009667573, -0.00...
1         [-0.4971033, -0.69848496, -0.43472102, 1.08629...
2         [-0.12381571, -0.5048019, 0.045582704, 0.07201...
3         [-0.33021298, -0.15855773, -0.21765107, 0.5498...
4         [-0.052870903, -0.7280239, -0.2046081, -0.3776...
                                ...                        
149395    [-0.3660654, -0.026480813, -0.021413691, 0.122...
149396    [-0.16405234, 0.15435684, 0.31223166, 0.093945...
149397    [0.39884868, -1.0513525, -0.47640738, 0.350791...
149398    [-0.24202009, 0.08661278, 0.03701636, -0.19218...
149399    [0.0041740537, -1.1935802, -0.10795422, 0.1160...
Name: preprocess, Length: 149400, dtype: object

در این مرحله، مجموعه‌داده را با استفاده از تابع train_test_split به دو بخش آموزشی و ارزیابی تقسیم می‌کنیم. داده‌ها به گونه‌ای تقسیم می‌شوند که ۸۰٪ برای آموزش و ۲۰٪ برای ارزیابی استفاده شوند. در اینجا:

X: بردارهای جملات (بردارهای تعبیه‌شده برای هر نظر)

y: برچسب‌های هدف (ستون recommendation_status)

In [35]:
# Convert sentence vectors to a NumPy array
X = np.array(sentence_vectors.to_list())

# Assuming 'df["recommendation_status"]' contains target labels
y = train_data["recommendation_status"].values

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

پس از آماده‌سازی داده‌ها و تقسیم آن‌ها به مجموعه‌های آموزشی و ارزیابی، زمان آموزش مدل فرا رسیده است. در این پروژه از رگرسیون لجستیک برای طبقه‌بندی احساسات استفاده می‌کنیم. مدل با استفاده از متد fit روی داده‌های آموزشی (X_train و y_train) آموزش خواهد دید.

In [37]:
# Initialize and train the Logistic Regression model
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

<hr>

## ارزیابی مدل

پس از آموزش مدل، زمان ارزیابی عملکرد آن است. در این مرحله از داده های ارزیابی (X_test) برای پیش بینی استفاده می کنیم و سپس با استفاده از تابع accuracy_score دقت مدل را محاسبه می کنیم. در نهایت دقت مدل را نمایش خواهیم داد.

In [39]:
# Make predictions on the test set
y_pred = logistic_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.6413654618473895


<hr>

## ایجاد تابع برای پیش بینی وضعیت توصیه  نظرات 

در این مرحله، تابعی به نام predict_recommendation ایجاد می‌کنیم که وضعیت توصیه یک نظر جدید را پیش‌بینی می‌کند. 

In [41]:
def predict_recommendation(comment):
    preprocessed_comment = preprocess_text(comment)
    sentence_vector_comment = sentence_vector(preprocessed_comment)
    X_comment = np.array([sentence_vector_comment])
    prediction = logistic_model.predict(X_comment)
    if prediction[0] == 2:
        return "no_idea"
    elif prediction[0] == 1:
        return "recommended"
    else:
        return "not_recommended"

In [43]:
new_comment = 'نخرید'
predict_recommendation(new_comment)

'not_recommended'

<hr>

### Github.com : RezaGooner

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0739d037-8289-409c-a01d-ddab9865ba9f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>